<a href="https://colab.research.google.com/github/cheul0518/DeepLearning/blob/main/Chapter1_MN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises

1. **Sigmoid neurons simulating perceptrons, part I**. Suppose we take all the weights and biases in a network of perceptrons, and multiply them by a positive constant, $c > 0$. Show that the behaviour of the network doesn't change.
  - A positive constant $c$ does not affect the sign of ($w\cdot x + b$) so the output won't change. In other words, the network doesn't change.

2. **Sigmoid neurons simulating perceptrons, part II**. Suppose we have the same setup as the last problem - a network of perceptrons. Suppose also that the overall input to the network of perceptrons has been chosen. We won't need the actual input value, we just need the input to have been fixed. Suppose the weights and biases are such that $w\centerdot x + b \neq 0$ for the input $x$ to any particular perceptron in the network. Now replace all the perceptrons in the network by sigmoid neurons, and multiply the weights and biases by a positive constant $c>0$. Show that in the limit as $c\to\infty$  the behaviour of this network of sigmoid neurons is exactly the same as the network of perceptrons. How can this fail when $w\centerdot x + b = 0$ for one of the perceptrons?
  - $\sigma(c(w\cdot x + b)) = \frac{1}{1+exp(-c(w\cdot x + b))}$.
  - As $c$ approaches $\infty$, $\sigma(c(w\cdot x + b)) = \frac{1}{1+exp(-\infty)} \approx 1$.
  - Like a perceptron of ($w$,$b$) with a fixed input $x$, Sigmoid neurons return $1$ In the limit as $c\to\infty$.
  - $\sigma(c(w\cdot x + b)) = \sigma(0) = \frac{1}{2}$, which is not output of perceptrons. So, this case fails to act like a perceptron. 

3. there is a way of determining the bitwise representation of a digit by adding an extra layer to the three-layer network above. The extra layer converts the output from the previous layer into a binary representation, as illustrated in the figure below. Find a set of weights and biases for the new output layer. Assume that the first 3 layers of neurons are such that the correct output in the third layer (i.e., the old output layer) has activation at least 0.99, and incorrect outputs have activation less than 0.01.
<div>
<img src="http://neuralnetworksanddeeplearning.com/images/tikz13.png">
</div>

In [127]:
"""
1. Create two matrices: activations from old output layer, and bitwise representaions of digits from new output layer.
  - Correct activation output's at least 0.99, and incorrect one's at most 0.01.

2. Sigmoid(wx+b) = new output. For simplicity, let's say b = 0.5. You can set b differently.
3. You can compute w as follows: wx + b = logit(new output) -> wx = logit(new output) - b -> w = (logit(new output) - b) inverse(x)
"""
import numpy as np
import random
from scipy.special import logit

def sigmoid(x):
  x = 1.0/(1.0+np.exp(-x))
  return x

# Digit per each column
act_old = np.zeros(shape=(10,10),dtype=float)
for i in range(act_old.shape[0]):
    for j in range(act_old.shape[1]):
        act_old[i][j] = -random.uniform(-0.01,0.0)  # I exclude 0 to avoid infinity computations.
for i in range(act_old.shape[0]):
    act_old[i][i] = random.uniform(0.99,1.0)  # I exclude 1 to avoid infinity computations.

# Bitwise representation of a digit per each row
bRep = np.zeros(shape=(10,4), dtype=float)
for i in range(10):
    print("digit: {}: {} ->".format(i, np.round(act_old[i]).astype(int)), end=' ')
    bRep[i] = np.asarray([j for j in format(i, '04b')])
    print(bRep[i].astype(int))

# Make some changes in bitwise representation matrix for fun: 0 -> (0 ~ 0.01) / 1 -> (0.99~1.0)
for i in range(bRep.shape[0]):
    for j in range(bRep.shape[1]):
        if bRep[i][j] == 0:
            bRep[i][j] = -random.uniform(-0.01,0.0)
        else:
            bRep[i][j] = random.uniform(0.99,1.0)    

# Apply logit function onto bit reprentation matrix, substract bias from logit(bRep), and multipy the inverse of activation matrix.
# Don't forget to transpose bRep and act_old.
bias = 0
w = np.dot((logit(bRep).T - bias), np.linalg.inv(act_old.T))
print("\nWeight matrix with integer numbers, when bias = {}\n{}".format(bias,w.astype(int)))
# Verify if we did it correctly
vrfy = sigmoid(np.dot(w,act_old.T) + bias)
for i in range(vrfy.shape[0]):
    for j in range(vrfy.shape[1]):
        if (vrfy[i][j] >= 0.99) and (vrfy[i][j] <= 1):
            vrfy[i][j] = 1
        elif (vrfy[i][j] <= 0.01) and (vrfy[i][j] >= 0):
            vrfy[i][j] = 0          
print("\nBit representation matrix with feedforward computation\n{}".format(vrfy.T))
print("\nThe verified bit representation matrix is the same as the original one. \nBut the above set of weights and biases's constantly changing because activation outputs are inconsistent.")

digit: 0: [1 0 0 0 0 0 0 0 0 0] -> [0 0 0 0]
digit: 1: [0 1 0 0 0 0 0 0 0 0] -> [0 0 0 1]
digit: 2: [0 0 1 0 0 0 0 0 0 0] -> [0 0 1 0]
digit: 3: [0 0 0 1 0 0 0 0 0 0] -> [0 0 1 1]
digit: 4: [0 0 0 0 1 0 0 0 0 0] -> [0 1 0 0]
digit: 5: [0 0 0 0 0 1 0 0 0 0] -> [0 1 0 1]
digit: 6: [0 0 0 0 0 0 1 0 0 0] -> [0 1 1 0]
digit: 7: [0 0 0 0 0 0 0 1 0 0] -> [0 1 1 1]
digit: 8: [0 0 0 0 0 0 0 0 1 0] -> [1 0 0 0]
digit: 9: [0 0 0 0 0 0 0 0 0 1] -> [1 0 0 1]

Weight matrix with integer numbers, when bias = 0
[[-5 -6 -4 -4 -5 -9 -4 -4  4  5]
 [-4 -6 -6 -6  5  5  7  4 -6 -5]
 [-4 -4  5  5 -5 -5  5  5 -6 -4]
 [-5  5 -5  5 -5  5 -5  4 -4  5]]

Bit representation matrix with feedforward computation
[[0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 1.]
 [0. 1. 1. 0.]
 [0. 1. 1. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 1.]]

The verified bit representation matrix is the same as the original one. 
But the above set of weights and biases's constantly changing because activation outp

4. Prove the assertion of the last parahraph: make a move $\Delta v$ in position so as to decrease $C$ as much as possible. This is equivalent to minimizing $\Delta C \approx \nabla C \cdot \Delta v$. We'll constrain the size of the move so that $||\Delta v||=\epsilon$ for some small fixed $\epsilon>0$. In other words, we want a move that is a small step of a fixed size, and we're trying to find the movement direction which decreases $C$ as much as possible. It can be proved that 
\
\
  *$\Delta v$ which minimizes $\nabla C \cdot\Delta v$ is $\Delta v = -\eta\nabla C$, where$||\Delta v|| = \epsilon$ and $\eta=\frac{\epsilon}{||\nabla C||}$*
  \
\
  So gradient descent can be viewed as a way of taking small steps in the direction which does the most to immediately decrease $C$. Hint: If you're not already familiar with the Cauchy-Schwarz inequality, you may find it helpful to familiarize yourself with it.
  - Cauchy-Schwarz inequality #1: $||u||\cdot||v|| \leq ||u\cdot v||$ if and only if $u$ and $v$ are linearly dependent.
  - Cauchy-Schwarz inequality #2: $||u||\cdot||v|| = ||u\cdot v||$ if and only if one of $u$ and $v$ is a scalar of the other. If this equality holds, and if $v \neq 0$, then $u = \frac{u\cdot v}{||v^2||}v$
  - Apply the first inequality rule to $\nabla C$ and $\Delta v$ because they are linearly dependent: $\Delta C \approx \nabla C \cdot \Delta v$
  - $||\nabla C||\cdot||\Delta v|| \leq ||\nabla C\cdot\Delta v|| \to -\nabla C\cdot\Delta v\leq ||\nabla C||\cdot||\Delta v|| \leq \nabla C\cdot\Delta v$
  - $min(\nabla C\cdot\Delta v) = -||\nabla C||\cdot||\Delta v|| = -\frac{\epsilon^2}{\eta}$
  - Use the second inequality rule in order to find $\Delta v$, because $||\Delta v|| = \eta||\nabla C||$
  - $\Delta v = \frac{(\Delta v\cdot \nabla C)}{||\nabla C^2||}\nabla C = \frac{-\frac{\epsilon^2}{\eta}}{\frac{\epsilon^2}{\eta^2}}\nabla C = -\eta\nabla C$.



5. I explained gradient descent when $C$ is a function of two variables, and when it's a function of more than two variables. What happens when $C$ is a function of just one variable? Can you provide a geometric interpretation of what gradient descent is doing in the one-dimensional case?
  - $C(v)$ with a single variable is on a plane. On the plane, Gradient is equal to the derivative of the function $C(v)$, which is the slope of a point. Gradient descent plays a role in finding a local minimum in the differentiable function $C$, by moving $v$ in the direction $\Delta C < 0$.
  - $y = C(v)$, and $x = v$ below. Let's say we have an initial coordinate of (2,0). Gradient descent will move $x$ toward  the local minimum at $x = \frac{1}{2}$
<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Polynomialdeg2.svg/1280px-Polynomialdeg2.svg.png" width="400">
</div>

6. An extreme version of gradient descent is to use a mini-batch size of just 1. That is, given a training input, $x$, we update our weights and biases according to the rules $w_k\to w' = w_k-\eta\frac{\partial C_x}{\partial w_k}$ and $b_l\to b'_l = b_l - \eta\frac{\partial C_x}{\partial b_l}$. Then we choose another training input, and update the weights and biases again. And so on, repeatedly. This procedure is known as online, on-line, or incremental learning. In online learning, a neural network learns from just one training input at a time (just as human beings do). Name one advantage and one disadvantage of online learning, compared to stochastic gradient descent with a mini-batch size of, say, 20.
  - Compared to SGD with mini batch size of 20, online learning is a lot slower in learning. Online learning performs slightly better in classifying the test set than the SGD when given the learning rate of 1.0. But, the SGD classifies the test set a lot better when the learning rate is 3.0. Online learning has an unique benefit that dynamically compute gradients while constantly receiving new parameters. 

In [ ]:
#@title 6-0. Download dataset and textbook codes
#Download the dataset from Nielsen's github
!wget -L https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz

#Download mnist_loader.py from Dobrzanski's github: his version's for python 3
!wget -L https://raw.githubusercontent.com/MichalDanielDobrzanski/DeepLearningPython/master/mnist_loader.py

#Download network2.py from Dobrzanski's github: his version's for python 3
!wget -L https://raw.githubusercontent.com/MichalDanielDobrzanski/DeepLearningPython/master/network.py

import mnist_loader
import network

In [ ]:
#@title 6-1. online learning: 94% accuracy (eta:1.0); 340seconds

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net = network.Network([784, 30, 10])
net.SGD(training_data,30, 1, 1.0, test_data=test_data)

In [ ]:
#@title 6-2. SGD with mini-batch size of 20: 94% accuracy; 253seconds
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net2 = network.Network([784, 30, 10])
net2.SGD(training_data,30, 20, 1.0, test_data=test_data)

7. Write out Equation $a' = \sigma(wa+b)$ in component form, and verify that it gives the same result as the rule $\sigma(z) \equiv \frac{1}{1+exp(\sum_j{-w_jx_j-b})}$ for computing the output of a sigmoid neuron.
  - $a$ and $b$ are vectors, and $w$ is a weight matrix.
  - In component form, $a_k' = \sigma(\sum_j{w_{kj}x_j-b_k}) = \frac{1}{1+exp(-\sum_j{w_{kj}x_j-b_k})} \equiv \frac{1}{1+exp(\sum_j{-w_jx_j-b})}$. 
  \
  So, equation $a'$ outputs the same result as $\sigma(z)$.

8. Try creating a network with just two layers - an input and an output layer, no hidden layer - with 784 and 10 neurons, respectively. Train the network using stochastic gradient descent. What classification accuracy can you achieve?
  - SGD without hidden layers has achieved 82% classification accuracy. The codes are below

In [ ]:
#@title 8-0. Download dataset and textbook codes
#Download the dataset from Nielsen's github
!wget -L https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz

#Download mnist_loader.py from Dobrzanski's github: his version's for python 3
!wget -L https://raw.githubusercontent.com/MichalDanielDobrzanski/DeepLearningPython/master/mnist_loader.py

#Download network2.py from Dobrzanski's github: his version's for python 3
!wget -L https://raw.githubusercontent.com/MichalDanielDobrzanski/DeepLearningPython/master/network.py

import mnist_loader
import network

In [ ]:
#@title 8-1. SGD with no hidden layers: 82% accuracy
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net_noH = network.Network([784, 10])
net_noH.SGD(training_data,30, 10, 3.0, test_data=test_data)